In [8]:

import pandas as pd
import networkx as nx
import os
import copy

In [9]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.4.2


In [10]:
from ema_workbench import (
    Policy,
    ema_logging,
    MultiprocessingEvaluator,
    save_results, 
    load_results,
)
from problem_formulation import get_model_for_problem_formulation



In [11]:
ema_logging.log_to_stderr(ema_logging.INFO)


<Logger EMA (DEBUG)>

### EPA141 Final Assignment: Exploratory Analysis Setup

This script configures and runs EMA Workbench experiments to explore the combined impact of external uncertainties and individual policy levers on model outcomes. We create a new function, named run_exploratory_experiments to transform levers into inputs and sample them in a similar fashion to the uncertainities, using Latin Hypercube. It is designed for an initial exploratory analysis phase to identify key drivers of the desired outcomes and to understand the space of possibilities. Literature supporting this approach (number of scenarios, exploring levers as factors):
- Bankes, S. (1993). Exploratory Modeling for Policy Analysis. Operations Research, 41(3), 435-449.
 - Bryant, B. P., & Lempert, R. J. (2010). Thinking inside the box: A participatory, computer-assisted approach to scenario discovery. Technological Forecasting & Social Change, 77(1), 34-49.
- Walker, W. E., Marchau, V. A. W. J., & Kwakkel, J. H. (2013). Uncertainty in the framework of Policy Analysis. In W. A. H. Thissen & W. E. Walker (Eds.), Public Policy Analysis:New Developments (pp. 215-261). Springer.
- Moallemi, E. A., Kwakkel, J., de Haan, F. J., & Bryan, B. A. (2020). Exploratory modeling for analyzing coupled human-natural systems under uncertainty. Global Environmental Change, 65, 102186.
- Course Material: "Final_Case_Study_Modeling_Steps.docx" (recommends 1000-3000 simulations).

This code is used to generate exploratory experimental data for different problem forumlations of interest to our clients (Group 7 and 21). Every time the problem forumation is changed, a new name should be given for the variable `filename` to ensure you avoid overwriting previous data. 

In [25]:

def run_exploratory_experiments(problem_formulation_id, num_scenarios, output_filename):
    """
    Sets up and runs exploratory experiments where both uncertainties and individual
    policy levers are sampled. Saves results using ema_workbench.save_results.

    Args:
        problem_formulation_id (int): The ID for the problem formulation to load.
        num_scenarios (int): The number of scenarios (experimental runs) to perform.
        output_filename (str): The filename for saving the results (e.g., "exploratory_results.tar.gz").
    """
    ema_logging.log_to_stderr(ema_logging.INFO)

    # --- 1. Load the Base Model Definition ---
    try:
        dike_model, _ = get_model_for_problem_formulation(problem_formulation_id)
        print(f"INFO: Loaded model for problem formulation {problem_formulation_id}.")
    except Exception as e:
        print(f"ERROR: Could not load model from problem_formulation.py: {e}")
        return

    # --- 2. Prepare Model for Exploratory Lever Sampling ---
    # For this exploratory run, individual policy levers are treated as factors to be sampled, similar to how uncertainties are sampled.
    
    exploratory_factors = [] # This list will hold all factors to be sampled.
    
    # Add original uncertainties
    if hasattr(dike_model, 'uncertainties'):
        exploratory_factors.extend(copy.deepcopy(dike_model.uncertainties))
    else:
        print("WARNING: dike_model has no 'uncertainties' attribute.")

    # Add original levers to the list of factors to be sampled
    # Ensure lever names do not clash with uncertainty names.
    original_levers = []
    if hasattr(dike_model, 'levers') and dike_model.levers:
        original_levers = copy.deepcopy(dike_model.levers)
        exploratory_factors.extend(original_levers)
        print(f"INFO: {len(original_levers)} policy levers will be sampled as factors.")
    else:
        print("WARNING: dike_model has no 'levers' or an empty list of levers.")

    if not exploratory_factors:
        print("ERROR: No uncertainties or levers found to explore. Aborting.")
        return

    # Temporarily assign this combined list to dike_model.uncertainties.
    # This instructs EMA Workbench to sample across both original uncertainties and levers.
    dike_model.uncertainties = exploratory_factors
    
    
    dike_model.levers = [] 

    print(f"INFO: Total number of factors to be sampled: {len(dike_model.uncertainties)}")

    # --- 3. Run Experiments ---
    print(f"INFO: Starting exploratory experiments with {num_scenarios} scenarios...")
    try:
        with MultiprocessingEvaluator(dike_model) as evaluator:
            # 'policies=None' ensures that the workbench does not generate separate policy bundles. Instead, variations in lever settings are part of the 'scenarios'.
            results_tuple = evaluator.perform_experiments(
                scenarios=num_scenarios, 
                policies=None 
            )
        print(f"INFO: Experiment run completed. {len(results_tuple[0])} scenarios were executed.")
        experiments, outcomes = results_tuple
    except Exception as e:
        print(f"ERROR: An error occurred during perform_experiments: {e}")
        return

    # --- 4. Save Results ---
    # Using ema_workbench.save_results is recommended as it preserves data structures.
    output_dir = "../experimental data"
    file_path = os.path.join(output_dir, output_filename)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir)
            print(f"INFO: Created output directory: {output_dir}")
        except OSError as e:
            print(f"ERROR: Could not create directory {output_dir}: {e}")
            return # Stop if directory cannot be created

    try:
        save_results(results_tuple, file_path)
        print(f"INFO: Exploratory results successfully saved to: {file_path}")
        # Uncomment the following lines to print the first few rows of the DataFrame
        # print (experiments) 
        # #print column names
        # print("INFO: Column names in the experiments DataFrame:")
        # print(experiments.columns)
        # print(pd.DataFrame(outcomes))
    except Exception as e:
        print(f"ERROR: Failed to save results to {file_path}: {e}")


In [29]:
#execute the function 
# Define experiment parameters
PROBLEM_FORMULATION_ID = 3 #helps our clients and to test the priorities of other actors
# Number of scenarios (runs), aiming for 1000-3000 as per literature and guidelines
# due to the function, the arrays get super messed up with around 2000 runs. 1000 is optimal. 
NUMBER_OF_SCENARIOS = 2000 
OUTPUT_FILENAME = "pf_3_exploratory_runs_levers_as_factors.tar.gz"

run_exploratory_experiments(PROBLEM_FORMULATION_ID, NUMBER_OF_SCENARIOS, OUTPUT_FILENAME)


INFO: Loaded model for problem formulation 3.
INFO: 31 policy levers will be sampled as factors.
INFO: Total number of factors to be sampled: 50
INFO: Starting exploratory experiments with 2000 scenarios...


/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")
/Users/precupada/decision_making_assignments/venv/lib/python3.13/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed -

INFO: Experiment run completed. 2000 scenarios were executed.
INFO: Exploratory results successfully saved to: ../experimental data/pf_3_exploratory_runs_levers_as_factors.tar.gz


### The following cells will be used to generate experimental runs with candidate policies


In [ ]:
# defining specific = template policies for later 
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [ ]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

In [ ]:
experiments, outcomes = results

In [ ]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)